# Benzinga-Nachrichten-Verarbeitung

## Setup up

In [1]:
use_colab = False
if use_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    cwd="/content/drive/MyDrive/NewsTrading/trading_bot"
    %cd /content/drive/MyDrive/NewsTrading/trading_bot
    %pip install -r requirements_clean.txt

In [2]:
%cd /gxfs_work/cau/sunms534/trading_bot/

/gxfs_work/cau/sunms534/trading_bot


/gxfs_work/cau/sunms534/.conda/envs/my_pytorch_env/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import os
import re
import pandas as pd
from src.config import config 
from tqdm.notebook import tqdm
tqdm.pandas()
import numpy as np

from IPython.display import clear_output

import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.WARNING)

from src.utils.dataframes import parallelize_dataframe, block_apply_factory
from src.preprocessing.news_parser import infer_author, filter_body, body_formatter

pd.set_option(
    'display.max_colwidth', 2000
)

## Import and merge news bodies from different sources

In [4]:
fnspid_news = pd.read_parquet(config.data.fnspid.raw_html_parsed)

In [5]:
bzg_news = pd.read_parquet(config.data.benzinga.raw_html_parsed)

In [6]:
bzg_news['intra_day_time'] = True

In [7]:
ddf = pd.concat([fnspid_news, bzg_news], axis=0)

In [8]:
print(f"{bzg_news.shape[0]}(bzg) + {fnspid_news.shape[0]} (fnspid) = {ddf.shape[0]}")

2086342(bzg) + 2491778 (fnspid) = 4578120


In [9]:
ddf = ddf.reset_index(drop=True)

## Author-Inferenz (obsolete, not useful)

In [10]:
ddf.memory_usage(deep=True).sum() / 1024**3

30.272836787626147

In [11]:
# Remove rows for which noo stock ticker is recorded
ddf = ddf[ddf.stocks != '']

Untersuche als nächstes die Behauptung, dass **PRNewswire** und **Businesswire** den gesamten Markt für Pressemeldungen in den USA kontrollieren. Wenn dem so ist, und sie nicht noch weitere, unwichtige Meldungen veröffentlichen, dann können wir einfach die Newsartikel nach diesen Autoren filtern und uns viel Arbeit ersparen.

In [12]:
%%time
ddf["inferred_author"] = None
ddf["inferred_author"] = ddf.body.progress_apply(infer_author)

  0%|          | 0/4577971 [00:00<?, ?it/s]

CPU times: user 7min 23s, sys: 4.82 s, total: 7min 28s
Wall time: 7min 26s


In [13]:
# value_counts for authors
auhtor_value_counts = pd.concat([ddf.author.value_counts().head(10), ddf.inferred_author.value_counts().head(10)], axis=1)

In [14]:
auhtor_value_counts.columns = ["author", "inferred_author"]
auhtor_value_counts

,author,inferred_author
Benzinga,1061214,NaN
PRNewswire,305720,586832.0
Globe Newswire,293466,476281.0
Business Wire,268561,296160.0
Newsfile,70877,NaN
ACCESSWIRE,62615,81107.0
"AB Digital, Inc.",9936,NaN
WebWire,6404,NaN
PRWeb,2617,NaN
News Direct,2080,NaN


In [15]:
ddf = ddf.drop(columns=["author"]).rename(columns={"inferred_author":"author"})

In [16]:
ddf["author"] = ddf['author'].astype("object").replace({pd.NA:np.nan})

In [17]:
# Legacy code... channels only present in benzinga news and only after 2017 aswell
# Contains 100k rows
# earnings_ddf = ddf[ddf.channels.apply(lambda x: "Earnings" in x)]
# value counts for authors of earnings reports (contrast to value counts of all news articles)
# earnings_ddf.author.value_counts().head(10)

In [18]:
# We dont really care about the author...
# ddf.drop(columns=['author'], inplace=True)

Hier sehen wir, dass es keine einzige Pressemeldung von **Business Wire** gibt, die mit *Earnings* gekennzeichnet sind. Trotzdem gibt es relevante *Earnings* reports von Business Wire. Dies habe ich kurz verifiziert...

Wie viele Nachrichten bleiben, wenn wir auf relevante Ticker filtern? Wir wollen nicht(!) - so ist es momentan - auf die momentane Russell 3k-Zusammensetzung filtern, denn wir wollen auch ungelistete bzw. ehemalige Russell-Aktien beachten.


**1. Full-Name-Discovery:**

Herausfinden des vollen Namens des Unternehmens für jeden Ticker, damit 1. der Text richtig geparst werden kann und 2. damit wir einen Anhaltspunkt für das Ticker-Grouping haben.


**2. Ticker-Filtering:**

Alle Ticker herausfiltern, die wir nicht brauchen. Wenn wir aber ein großes Aktienuniversum (mit inzwischen ungelisteten Aktien) benutzen, werden wir fast alle Nachrichten behalten können. Allerdings lassen sich so Fehlerhafte Nachrichten/Ticker etc. herausfiltern.


**3. Ticker-Grouping:**

Was machen wir, wenn wir mehrerer Aktiengattungen für ein Unternehmen haben? Z.B. Vorzugs- und Stammaktien. Wir können i.A. die Stammaktie nehmen, da diese normalerweise ein höheres Handelsvolumen aufweist. D.h. wir bilden alle Ticker der Benzinga-Nachrichten auf den Ticker der Stammaktie ab.


**4. Firmennamen-Nachrichtenkörper-Verifikation:**

Da Ticker wiederverwendet werden können bzw. sich verändern können wollen wir sicherstellen, dass der Unternehmensname im Nachrichtenkörper vorkommt. Bzw. generell ist das eine gute Datensäuberungs-Maßnahme. Einerseits verhindern wir,
dass später Aktienkurse einer falschen Aktie zugeordnet wird. Andererseits werden dadurch evtl. auch weniger seriöse Nachrichten herausgefiltert, die nicht
die Kontaktadresszeile des Unternehmens am Ende besitzen, in dem der vollständige Unternehmensname vorkommt.

In [19]:
mapper = pd.read_parquet(config.data.shared.ticker_name_mapper)

In [20]:
ddf = ddf[ddf.stocks.isin(mapper.index.to_list())]
ddf["company_name"] = ddf.stocks.progress_map(lambda x: mapper.company_names.loc[x]).astype(str)
ddf["short_name"] = ddf.stocks.progress_map(lambda x: mapper.short_name.loc[x]).astype(str)
print(f"Es verbleiben {ddf.shape[0]} Nachrichten, für die wir den Ticker zu einem Firmennamen aufgelösen konnten.")

  0%|          | 0/3223598 [00:00<?, ?it/s]

  0%|          | 0/3223598 [00:00<?, ?it/s]

Es verbleiben 3223598 Nachrichten, für die wir den Ticker zu einem Firmennamen aufgelösen konnten.


In [21]:
ddf.company_name

0                  Agilent Technologies, Inc.
1                  Agilent Technologies, Inc.
2                  Agilent Technologies, Inc.
3                  Agilent Technologies, Inc.
4                  Agilent Technologies, Inc.
                          ...                
4578114              Toyota Motor Corporation
4578115                     Kirby Corporation
4578116       Chindata Group Holdings Limited
4578117    Orbital Infrastructure Group, Inc.
4578118                     ImmunityBio, Inc.
Name: company_name, Length: 3223598, dtype: object

### Duplikate Entfernen

In [22]:
%%time
samples_before = ddf.shape[0]
ddf = ddf.drop_duplicates()
samples_after = ddf.shape[0]
print(f"{samples_before=}, {samples_after=}")

samples_before=3223598, samples_after=3197979
CPU times: user 39.4 s, sys: 4.89 s, total: 44.3 s
Wall time: 44.4 s


### Firmennamen-Nachrichtenkörper-Verifikation

In [23]:
def verify_company_name_in_body(df):
    return df.apply(lambda x: bool(re.search(x["short_name"],
                                             x.title + x["body"].replace("( )*\n( )*", " "),
                                             re.IGNORECASE)),
                    axis=1)

In [24]:
%%time
mask = parallelize_dataframe(ddf, verify_company_name_in_body, n_cores=os.cpu_count())

print(f"Around {len(ddf.stocks.unique())} stocks before filtering and {len(ddf[mask].stocks.unique())} after")

n_cores=32, df.shape=(3197979, 9)


/gxfs_work/cau/sunms534/.conda/envs/my_pytorch_env/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
parallelize_data: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:33<00:00,  1.05s/it]


Around 11383 stocks before filtering and 10322 after
CPU times: user 14.2 s, sys: 17.9 s, total: 32 s
Wall time: 44 s


In [25]:
# Filter out faulty news, for which the company name doesn't occurr in the news body
ddf = ddf[mask]

In [26]:
print(f"{ddf.shape[0]} stocks left.")

2667193 stocks left.


Bis zu 5k Nachrichten pro Firma, z.B. AT&T, was in 13 Jahren ca. einer Nachricht pro Tag entspricht. Wir wollen nicht das eine Firma mit vielen Junk-Nachrichten das Modell dominiert. Wobei das Modell hoffentlich dann auch die Junk-Nachrichten als solche erkennt und ignoriert. Eher wichtig noch einen `staleness`-Faktor, also wie ähnlich die Nachricht zu Vorhergegangenen ist (i.e. Nachrichten desselben Tages oder derselben Woche).

Kategorisieren von Nachrichten (mit Text2Topic, wie Salbrechter?) und eliminieren von Business/Strategic etc.
Im Falle von Text2Topic, versuche Estimates des Unternehmens von Dritten zu unterscheiden.

Wichtig!!! Unterscheide zwischen LERN-Phase und PRODUKTIONS-Phase.
Wir können z.B. CLS-Token in der Produktions-Phase vergleichen, in der Lern-Phase aber noch nicht.

Text2Vec -> Business category evtl. entfernen-> Intrastock variance average

## Make reduced ticker name mapping 

In [27]:
ticker_name_mapper = pd.read_parquet(config.data.shared.ticker_name_mapper)

ticker_name_mapper_reduced = ddf[["stocks", "company_name", "short_name"]].drop_duplicates(keep="first")

ticker_name_mapper_reduced.to_parquet(config.data.shared.ticker_name_mapper_reduced)

print(f"From {ticker_name_mapper.shape[0]} to {ticker_name_mapper_reduced.shape[0]} tickers (reduced)")

From 11383 to 10322 tickers (reduced)


## Parsing News Bodies


In [28]:
from src.utils.time import convert_timezone
ddf["time"] = ddf["time"].progress_map(lambda x: convert_timezone(pd.to_datetime(x)))

  0%|          | 0/2667193 [00:00<?, ?it/s]

In [29]:
sample = ddf

In [30]:
sample.columns

Index(['time', 'stocks', 'title', 'body', 'intra_day_time', 'channels',
       'author', 'company_name', 'short_name'],
      dtype='object')

In [31]:
n_cores = 32
n_cores = n_cores if n_cores else os.cpu_count()

In [ ]:
ddf["parsed_body"] = parallelize_dataframe(sample, block_apply_factory(filter_body, axis=1), n_cores=n_cores)

n_cores=32, df.shape=(2667193, 9)


/gxfs_work/cau/sunms534/.conda/envs/my_pytorch_env/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
parallelize_data:   0%|                                                                                                                          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/83350 [00:00<?, ?it/s]

  0%|          | 0/83350 [00:00<?, ?it/s]

  0%|          | 0/83350 [00:00<?, ?it/s]

...

  0%|          | 0/83349 [00:00<?, ?it/s]

  0%|          | 0/83349 [00:00<?, ?it/s]

  0%|          | 0/83349 [00:00<?, ?it/s]

  0%|          | 0/83349 [00:00<?, ?it/s]

  0%|          | 0/83349 [00:00<?, ?it/s]

In [ ]:
ddf.to_parquet(config.data.news.cleaned)

In [ ]:
### Analysis

In [ ]:
ddf.columns

In [ ]:
ddf.company_name.tail(10)

In [ ]:
ddf[["body", "parsed_body"]].tail(10).iloc[0:1, :]

In [ ]:
ddf[["body", "parsed_body"]].tail(10).iloc[0,1]

## Filtern von Newstiteln

## Voranstellen von gefilterten Newstiteln an Nachrichtenkörper

# Analyse

## Durschnittlichen Tokenlänge

### Timedeltas zwischen Nachrichtenmeldungen

In [ ]:
import plotly.express as px


In [ ]:
ddf = pd.read_parquet(config.data.news.cleaned)

In [ ]:
ddf = ddf.sort_values("time")
tmp = ddf[["time", "stocks"]]
#### Adding timedeltas to the data frame
news_timedeltas = tmp.groupby("stocks").transform(lambda x: x.diff())
ddf.loc[:, "timedelta"] = news_timedeltas.time.fillna(pd.Timedelta(days=100))

In [ ]:
news_timedeltas = ddf.timedelta
news_timedeltas.iloc[0].components
timedelta_in_minutes = news_timedeltas.apply(lambda x: x.total_seconds() / 60)
px.histogram(timedelta_in_minutes)